# Data Science Unit 2 Sprint Challenge 3

## Logistic Regression and Beyond

In this sprint challenge you will fit a logistic regression modeling the probability of an adult having an income above 50K. The dataset is available at UCI:

https://archive.ics.uci.edu/ml/datasets/adult

Your goal is to:

1. Load, validate, and clean/prepare the data.
2. Fit a logistic regression model
3. Answer questions based on the results (as well as a few extra questions about the other modules)

Don't let the perfect be the enemy of the good! Manage your time, and make sure to get to all parts. If you get stuck wrestling with the data, simplify it (if necessary, drop features or rows) so you're able to move on. If you have time at the end, you can go back and try to fix/improve.

### Hints

It has a variety of features - some are continuous, but many are categorical. You may find [pandas.get_dummies](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html) (a method to one-hot encode) helpful!

The features have dramatically different ranges. You may find [sklearn.preprocessing.minmax_scale](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.minmax_scale.html#sklearn.preprocessing.minmax_scale) helpful!

In [220]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

## Part 1 - Load, validate, and prepare data

The data is available at: https://archive.ics.uci.edu/ml/datasets/adult

Load it, name the columns, and make sure that you've loaded the data successfully. Note that missing values for categorical variables can essentially be considered another category ("unknown"), and may not need to be dropped.

You should also prepare the data for logistic regression - one-hot encode categorical features as appropriate.

In [221]:
# TODO - your work!
df = pd.read_csv("https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data",header=None,\
                 names =["age","workclass","fnlwgt","education","education_num","marital_status","occupation"\
                         ,"relationship","race","sex","capital_gain","capital_loss","hours-per-week"\
                         ,"native_country","Income"] )

In [222]:
df.head()

,age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours-per-week,native_country,Income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [266]:
df.isna().sum()

age               0
workclass         0
fnlwgt            0
education         0
education_num     0
marital_status    0
occupation        0
relationship      0
race              0
sex               0
capital_gain      0
capital_loss      0
hours-per-week    0
native_country    0
Income            0
dtype: int64

In [223]:
df["workclass"].unique()

array([' State-gov', ' Self-emp-not-inc', ' Private', ' Federal-gov',
       ' Local-gov', ' ?', ' Self-emp-inc', ' Without-pay',
       ' Never-worked'], dtype=object)

In [274]:
df["native_country"] = df["native_country"].replace(" ?", "unknown_country_of_origin")
df["workclass"] = df["workclass"].replace(" ?", "unknown_workclass")
df["occupation"] = df["occupation"].replace(" ?", "unknown_occupation")

In [275]:
df["sex"] = df["sex"]
df["sex"] = df["sex"].replace(" Male",1)
df["sex"] = df["sex"].replace(" Female",0)

In [276]:
unique_race

array([' White', ' Black', ' Asian-Pac-Islander', ' Amer-Indian-Eskimo',
       ' Other'], dtype=object)

In [277]:
df["Income"][0]
df["Income"] = df["Income"].replace(" <=50K",0)
df["Income"] = df["Income"].replace(" >50K",1)

In [278]:
occupation = pd.DataFrame()
unique_occupation = df["occupation"].unique()
for i in unique_occupation:
    occupation[i] = df["occupation"].apply(lambda x : 1 if i == x else 0)

In [279]:
native_country = pd.DataFrame()
unique_countries = df["native_country"].unique()
for i in unique_countries:
    native_country[i] = df["native_country"].apply(lambda x : 1 if i == x else 0)

In [280]:
workclass = pd.DataFrame()
unique_workclass = df["workclass"].unique()
for i in unique_workclass:
    workclass[i] = df["workclass"].apply(lambda x : 1 if i == x else 0)

In [281]:
marital_status = pd.DataFrame()
unique_status = df["marital_status"].unique()
for i in unique_status:
    marital_status[i] = df["marital_status"].apply(lambda x : 1 if i == x else 0)

In [282]:
race = pd.DataFrame()
unique_race = df["race"].unique()
for i in unique_race:
    race[i] = df["race"].apply(lambda x : 1 if i ==x else 0)

In [283]:
unique_occupation

array([' Adm-clerical', ' Exec-managerial', ' Handlers-cleaners',
       ' Prof-specialty', ' Other-service', ' Sales', ' Craft-repair',
       ' Transport-moving', ' Farming-fishing', ' Machine-op-inspct',
       ' Tech-support', 'unknown_occupation', ' Protective-serv',
       ' Armed-Forces', ' Priv-house-serv'], dtype=object)

In [284]:
scaled_numeric = df[["age","education_num","capital_gain","capital_loss", "hours-per-week"]]
scaled_numeric = StandardScaler().fit_transform(scaled_numeric)
scaled_numeric = pd.DataFrame(scaled_numeric)
scaled_numeric.columns = ["age","education_num","capital_gain","capital_loss", "hours-per-week"]

In [285]:
df2 = pd.concat([df.drop(["native_country","workclass","marital_status","race","sex", "age","education_num",\
                          "capital_gain","capital_loss", "hours-per-week","occupation"],axis=1),\
                 native_country,workclass,marital_status,race, scaled_numeric,occupation],axis=1)

## Part 2 - Fit and present a Logistic Regression

Your data should now be in a state to fit a logistic regression. Use scikit-learn, define your `X` (independent variable) and `y`, and fit a model.

Then, present results - display coefficients in as interpretible a way as you can (hint - scaling the numeric features will help, as it will at least make coefficients more comparable to each other). If you find it helpful for interpretation, you can also generate predictions for cases (like our 5 year old rich kid on the Titanic) or make visualizations - but the goal is your exploration to be able to answer the question, not any particular plot (i.e. don't worry about polishing it).

It is *optional* to use `train_test_split` or validate your model more generally - that is not the core focus for this week. So, it is suggested you focus on fitting a model first, and if you have time at the end you can do further validation.

In [286]:
# TODO - your work!
X_train, X_test, y_train, y_test = train_test_split(df2.drop(["Income","relationship", \
                                                              "education","fnlwgt" ],axis=1), df2["Income"] )
log_reg = LogisticRegression().fit(X_train, y_train)
#predictions = np.array(log_reg.predict(X_test)).reshape(-1,1)
y_test= np.array(y_test).reshape(-1,1)
print(log_reg.score(X_test, y_test))
b = np.array(X_train.columns)
coef = (log_reg.coef_).flatten()
z = zip(coef,b)
z = sorted(z, key=lambda x: x[0])
for i in z:
    print(tuple(i))

0.8457191991155878
(-1.4450298468522036, ' Priv-house-serv')
(-1.2809294575872798, ' Never-married')
(-1.1523207211839943, ' Farming-fishing')
(-1.0780808507669937, ' Dominican-Republic')
(-1.0755863131506627, ' Vietnam')
(-0.9927825175195835, ' Widowed')
(-0.9472720604784176, ' Columbia')
(-0.9289846009759934, ' Separated')
(-0.8531125834368354, ' Divorced')
(-0.8281875308881574, ' Without-pay')
(-0.7837440760578854, ' Guatemala')
(-0.7771940486169373, ' Other-service')
(-0.7349510906745685, ' Amer-Indian-Eskimo')
(-0.6293077125025529, ' Handlers-cleaners')
(-0.619732424597482, ' Other')
(-0.6063911533208567, ' Married-spouse-absent')
(-0.5303421638171203, ' Greece')
(-0.5083516124557407, ' Self-emp-not-inc')
(-0.5047983634919838, ' South')
(-0.4953940055911802, ' Outlying-US(Guam-USVI-etc)')
(-0.4699560003045341, ' Trinadad&Tobago')
(-0.4694865796793908, ' Thailand')
(-0.45728221850031026, ' Nicaragua')
(-0.4270836522610185, ' State-gov')
(-0.38668786818777845, ' China')
(-0.37395763

# dropping capital_gain

In [265]:
# TODO - your work!
X_train, X_test, y_train, y_test = train_test_split(df2.drop(["Income","relationship", \
                                                              "education","fnlwgt", "capital_gain", "capital_loss" ],axis=1), df2["Income"] )
log_reg = LogisticRegression().fit(X_train, y_train)
#predictions = np.array(log_reg.predict(X_test)).reshape(-1,1)
y_test= np.array(y_test).reshape(-1,1)
print(log_reg.score(X_test, y_test))
b = np.array(X_train.columns)
coef = (log_reg.coef_).flatten()
z = zip(coef,b)
z = sorted(z, key=lambda x: x[0])
for i in z:
    print(tuple(i))

0.8283994595258568
(-1.4713189031452398, ' Priv-house-serv')
(-1.2494792179911751, ' Never-married')
(-1.109533637647043, ' Columbia')
(-0.9325948667172679, ' Farming-fishing')
(-0.8860953888975408, ' Separated')
(-0.8655464885684683, ' Divorced')
(-0.8288233467712248, ' Other-service')
(-0.8265452341548828, ' Widowed')
(-0.8193580758266263, ' Married-spouse-absent')
(-0.7989109366255953, ' Without-pay')
(-0.7485375785689419, ' Laos')
(-0.7273139064364982, ' Dominican-Republic')
(-0.6294730192838676, ' Other')
(-0.5736245229142339, ' Amer-Indian-Eskimo')
(-0.5735744305091174, ' Vietnam')
(-0.5623478338122744, ' Outlying-US(Guam-USVI-etc)')
(-0.5463771820790465, ' Handlers-cleaners')
(-0.5307813596766675, ' Armed-Forces')
(-0.5236428934961476, ' China')
(-0.5160934643506764, ' Peru')
(-0.48457460256319596, ' Self-emp-not-inc')
(-0.48363806960390354, ' Thailand')
(-0.477821926834758, ' South')
(-0.4756542813201754, ' ?')
(-0.4512042599430562, ' State-gov')
(-0.4333068466013079, ' Greece'

## Part 3 - Analysis, Interpretation, and Questions

### Based on your above model, answer the following questions

1. What are 3 features positively correlated with income above 50k?
2. What are 3 features negatively correlated with income above 50k?
3. Overall, how well does the model explain the data and what insights do you derive from it?

*These answers count* - that is, make sure to spend some time on them, connecting to your analysis above. There is no single right answer, but as long as you support your reasoning with evidence you are on the right track.

Note - scikit-learn logistic regression does *not* automatically perform a hypothesis test on coefficients. That is OK - if you scale the data they are more comparable in weight.

### Match the following situation descriptions with the model most appropriate to addressing them

In addition to logistic regression, a number of other approaches were covered this week. Pair them with the situations they are most appropriate for, and briefly explain why.

Situations:
1. You are given data on academic performance of primary school students, and asked to fit a model to help predict "at-risk" students who are likely to receive the bottom tier of grades.
2. You are studying tech companies and their patterns in releasing new products, and would like to be able to model and predict when a new product is likely to be launched.
3. You are working on modeling expected plant size and yield with a laboratory that is able to capture fantastically detailed physical data about plants, but only of a few dozen plants at a time.

Approaches:
1. Ridge Regression
2. Quantile Regression
3. Survival Analysis

## Answers
1. What are 3 features positively correlated with income above 50k?
    - hours per week
    - being married to a government employee, AF or civil
    - number of years of education

2. What are 3 features negatively correlated with income above 50k?
    - Working in the private service industry
    - Never marrying or being widowed, or having an absent spouse
    - Emmigrating from South American / Carribean countries
3. Overall, how well does the model explain the data and what insights do you derive from it?
    - This model has an accuracy of around 85%, indicating a pretty good model. I think this model would better predict a different income threshold. 50k is around median for America. I think there is probably a lot of noise around the average. Maybe adding a current city/zipcode or living in a city vs rural area would improve the model.
    - I thought race would be a bigger factor, but it is a negative coefficient for black and white and Asian-Pac-Islander.
    - even though capital_gain/loss seems like it would have too much predictive power, removing it does not change the accuracy of the model much. The Jamaican factor flipped from positive to negative
    - I say all this, but maybe the model is more random than i thought. Cambodians have a large positive coefficient, but vietnamese people,laosians, and natives of thailand have a large negative coefficient. Maybe a feature for immigrant network effects would improve accuracy
    - Local government employees coef is negative but government employees coef is positive.

## Answers 2

1. I think Quantile Regression would be the best approach. Quant-reg would give us the best chance at not overestimating a student. We would understimate a lot of students, but we would be less likely to miss struggling students.
2. I think survival analysis would be best for this situation. Suvival Analysis is used to model timelines and changes. By tracking the amount of time a company has taken to launch products, we could predict their ability to launch a product on time.
3. Ridge Regression is best for this scenario. With a small sample size and a large number of features, we need a way to regularize(keep one feature from dominating the equation) our features and ridge regression is the best tool. 

**TODO - your answers!**